# Renewable Energy Facilities - Distributed Energy Resources - Summarize

In [2]:
import os
import requests
import pandas as pd
import geopandas as gpd
import sys
import datetime
sys.path.append("../morpc-common/")
import morpc
import frictionless

## Gather Inputs

In [3]:
data, resource, schema = morpc.frictionless_load_data('../morpc-renewenergyfacilities-standardize/output_data/distributed_energy_resources.resource.yaml')

morpc.load_frictionless_data | INFO | Loading Frictionless Resource file at location ..\morpc-renewenergyfacilities-standardize\output_data\distributed_energy_resources.resource.yaml
morpc.load_frictionless_data | INFO | Loading data, resource file, and schema from their source locations
morpc.load_frictionless_data | INFO | --> Data file: ..\morpc-renewenergyfacilities-standardize\output_data\distributed_energy_resources.csv
morpc.load_frictionless_data | INFO | --> Resource file: ..\morpc-renewenergyfacilities-standardize\output_data\distributed_energy_resources.resource.yaml
morpc.load_frictionless_data | INFO | --> Schema file: ..\morpc-renewenergyfacilities-standardize\output_data\distributed_energy_resources.schema.yaml
morpc.load_frictionless_data | INFO | Loading data.
frictionless_cast_field_types | INFO | Casting field geometry as type geojson.
frictionless_cast_field_types | INFO | Fieldname geometry as geojson. Attempting to convert to geometry.
frictionless_cast_field_type

In [4]:
der_data = gpd.GeoDataFrame(data).set_crs('EPSG:3735')

In [5]:
juris_geos = morpc.load_spatial_data('../morpc-geos-collect/output_data/morpc-geos.gpkg', layerName='JURIS')

morpc.load_spatial_data | INFO | Loading spatial data from location: ../morpc-geos-collect/output_data/morpc-geos.gpkg
morpc.load_spatial_data | INFO | Driver name is unspecified.  Will attempt to infer driver from file extension in source path.
morpc.load_spatial_data | INFO | Selecting driver GPKG based on file extension .gpkg
morpc.load_spatial_data | INFO | Reading spatial data...


C:\Users\jinskeep\AppData\Local\miniconda3\envs\morpc_env\Lib\site-packages\pyogrio\raw.py:137: RuntimeWarning: driver GPKG does not support open option DRIVER
  result = ogr_read(


In [6]:
juris_geos['JURISNAME'] = [f"{x} {y}" for x, y in zip(juris_geos['NAME'], juris_geos['MUNITYPE'])]

In [7]:
county_geos = morpc.load_spatial_data('../morpc-geos-collect/output_data/morpc-geos.gpkg', layerName='COUNTY')

morpc.load_spatial_data | INFO | Loading spatial data from location: ../morpc-geos-collect/output_data/morpc-geos.gpkg
morpc.load_spatial_data | INFO | Driver name is unspecified.  Will attempt to infer driver from file extension in source path.
morpc.load_spatial_data | INFO | Selecting driver GPKG based on file extension .gpkg
morpc.load_spatial_data | INFO | Reading spatial data...


C:\Users\jinskeep\AppData\Local\miniconda3\envs\morpc_env\Lib\site-packages\pyogrio\raw.py:137: RuntimeWarning: driver GPKG does not support open option DRIVER
  result = ogr_read(


In [8]:
county_geos['COUNTYFP'] = [f'39{x}' for x in county_geos['COUNTYFP']]

In [9]:
county_geos['COUNTYNAME'] = county_geos['COUNTYFP'].map(morpc.CONST_COUNTY_ID_TO_NAME)

In [10]:
region_geos = morpc.load_spatial_data('../morpc-geos-collect/output_data/morpc-geos.gpkg', layerName='REGION15')

morpc.load_spatial_data | INFO | Loading spatial data from location: ../morpc-geos-collect/output_data/morpc-geos.gpkg
morpc.load_spatial_data | INFO | Driver name is unspecified.  Will attempt to infer driver from file extension in source path.
morpc.load_spatial_data | INFO | Selecting driver GPKG based on file extension .gpkg
morpc.load_spatial_data | INFO | Reading spatial data...


C:\Users\jinskeep\AppData\Local\miniconda3\envs\morpc_env\Lib\site-packages\pyogrio\raw.py:137: RuntimeWarning: driver GPKG does not support open option DRIVER
  result = ogr_read(


In [11]:
def join_points_polygons(points, polygons, index_col, crs, how, predicate):
    if crs != None:
        print(f"INFO | Converting points crs and polygons crs to user defined option: {crs}.")
        points = points.to_crs(crs)
        polygons = polygons.to_crs(crs)
    if points.crs != polygons.crs:
        print(f"WARNING | CRS do not match. Converting points crs to match polygons crs: {polygons.crs.name}")
        points = points.to_crs(polygons.crs)
        
    try:
        df = points.sjoin(polygons, how=how, predicate=predicate)
    except RuntimeError as r:
        print(f"ERROR | Unable to spatially join points and polygons {r}.") 
    finally:
        print(f"INFO | Joined points and polygons.")

    print(f"INFO | Indexing by {index_col} from polygons.")
    data = df.copy()
    data = data.loc[~data[index_col].isna()] ## Remove rows with empty join
    return(data)

def calc_counts_sums(data, index_col, year_col, type_col, sum_col):
    print(f"INFO | Calculating sums for {sum_col} and counts for {type_col}")
    data = data.copy()
    data['count'] = 1
    count_col = 'count'
    data['total'] = data[sum_col]
    data = data[[index_col, year_col, type_col, 'count', 'total']].groupby([index_col, year_col, type_col]).sum().reset_index()
    data = data.set_index([index_col, year_col])
    data = data.pivot(columns = [type_col]).fillna(0)
    data.columns = data.columns.map("_".join)
    columns = data.columns # Store columns to add zero observations later
    data = data.reset_index()
    return(data, columns)

def add_zeros_for_non_obs(data, index_col, year_col, columns):
    print("INFO | Adding zeros to empty observations.")
    # Add observations that have zeros so all combinations of index and year have a observation for all types.
    accounted_place = data[index_col].unique()	
    first_year = data[year_col].min()
    last_year = pd.Series([x for x in data[year_col] if x < 9999]).max()

    data_zeros = data.copy()
    for year in range(int(first_year), int(last_year)+1, 1):  # range stops at one less than the second parameter, hence the +1
        for place in accounted_place:
            if data_zeros[(data_zeros[year_col] == year) & (data_zeros[index_col] == place)].shape[0] == 0:
                entry = {
                    year_col: year,
                    index_col: place,
                }
                for c in columns:
                    entry[c] = 0
                
                data_zeros = pd.concat([data_zeros, pd.DataFrame.from_dict(entry, orient='index').transpose()], ignore_index=True)
    return (data_zeros)

def rejoin_preserve_col(data, df, index_col, year_col, preserve_cols):
    print(f'INFO | Rejoining {preserve_cols} by {index_col}')
    # Rejoin columns to preserve
    data = data.set_index(index_col).join(df[list(pd.core.common.flatten([index_col, preserve_cols]))].drop_duplicates().set_index(index_col), how='left')
    data = data.reset_index()
    data = data.sort_values([index_col, year_col])
    return(data)

def pivot_wide_to_long(data, index_col, preserve_cols, year_col, type_col):
    print("INFO | Pivoting to long format")
    data = data.melt(id_vars=list(pd.core.common.flatten([index_col, preserve_cols, year_col]))).rename(columns={'variable':type_col})
    data = data.sort_values([index_col, type_col, year_col])
    return(data)

def summarize_points_in_polygons(points, polygons, index_col, year_col, type_col, preserve_cols, sum_col, crs=None, how = 'left', predicate = 'intersects'):

    data = join_points_polygons(points, polygons, index_col, crs, how = 'left', predicate='within')
    
    count_sums, columns = calc_counts_sums(data, index_col, year_col, type_col, sum_col)
    
    data_zeros = add_zeros_for_non_obs(count_sums, index_col, year_col, columns) 
    
    wide = rejoin_preserve_col(data_zeros, data, index_col, year_col, preserve_cols)
    
    long = pivot_wide_to_long(wide, index_col, preserve_cols, year_col, type_col)

    return(wide, long)

In [12]:
juris = {'points':der_data,
        'polygons':juris_geos,
        'index_col':"JURISNAME",
        'year_col':"YEAR",
        'type_col':"FuelType",
        'preserve_cols':'GEOIDFQ',
        'sum_col':"ReportedCapacity_kW",}

In [13]:
county = {'points':der_data,
        'polygons':county_geos,
        'index_col':"COUNTYNAME",
        'year_col':"YEAR",
        'type_col':"FuelType",
        'preserve_cols':'GEOIDFQ',
        'sum_col':"ReportedCapacity_kW",}

In [14]:
region = {'points':der_data,
        'polygons':region_geos,
        'index_col':"NAME",
        'year_col':"YEAR",
        'type_col':"FuelType",
        'preserve_cols':'GEOIDFQ',
        'sum_col':"ReportedCapacity_kW"}

In [15]:
OUTPUT_DIR = './output_data'
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [16]:
juris_wide, juris_long = summarize_points_in_polygons(**juris)
juris_wide.to_csv(f'{OUTPUT_DIR}/morpc-der-juris-wide.csv', index=False)
juris_long.to_csv(f'{OUTPUT_DIR}/morpc-der-juris-long.csv', index=False)                

INFO | Joined points and polygons.
INFO | Indexing by JURISNAME from polygons.
INFO | Calculating sums for ReportedCapacity_kW and counts for FuelType
INFO | Adding zeros to empty observations.
INFO | Rejoining GEOIDFQ by JURISNAME
INFO | Pivoting to long format


In [17]:
county_wide, county_long = summarize_points_in_polygons(**county)
county_wide.to_csv(f'{OUTPUT_DIR}/morpc-der-county-wide.csv', index=False)
county_long.to_csv(f'{OUTPUT_DIR}/morpc-der-county-long.csv', index=False)       

INFO | Joined points and polygons.
INFO | Indexing by COUNTYNAME from polygons.
INFO | Calculating sums for ReportedCapacity_kW and counts for FuelType
INFO | Adding zeros to empty observations.
INFO | Rejoining GEOIDFQ by COUNTYNAME
INFO | Pivoting to long format


In [18]:
region_wide, region_long = summarize_points_in_polygons(**region)
region_wide.to_csv(f'{OUTPUT_DIR}/morpc-der-region-wide.csv', index=False)
region_long.to_csv(f'{OUTPUT_DIR}/morpc-der-region-long.csv', index=False)    

INFO | Joined points and polygons.
INFO | Indexing by NAME from polygons.
INFO | Calculating sums for ReportedCapacity_kW and counts for FuelType
INFO | Adding zeros to empty observations.
INFO | Rejoining GEOIDFQ by NAME
INFO | Pivoting to long format


In [19]:
region_long['SUMLEVEL'] = 'M01'
region_long['METRIC'] = ['Facilities' if x.startswith('count_') else 'Capacity' for x in region_long['FuelType']]
region_long['FUEL_TYPE'] = [x.split("_")[-1] for x in region_long['FuelType']]
region_long[['NAME', 'GEOIDFQ', 'SUMLEVEL', 'YEAR', 'METRIC', 'FUEL_TYPE', 'VALUE']]
region_long['VALUE'] = [float(x) for x in region_long['VALUE']]
region_long = region_long[['NAME', 'GEOIDFQ', 'SUMLEVEL', 'YEAR', 'METRIC', 'FUEL_TYPE', 'VALUE']]

In [20]:
county_long['SUMLEVEL'] = '050'
county_long['METRIC'] = ['Facilities' if x.startswith('count_') else 'Capacity' for x in county_long['FuelType']]
county_long['FUEL_TYPE'] = [x.split("_")[-1] for x in county_long['FuelType']]
county_long = county_long.rename(columns = {'COUNTYNAME':'NAME','Year':'YEAR', 'value':"VALUE"})
county_long[['NAME', 'GEOIDFQ', 'SUMLEVEL', 'YEAR', 'METRIC', 'FUEL_TYPE', 'VALUE']]
county_long['VALUE'] = [float(x) for x in county_long['VALUE']]
county_long = county_long[['NAME', 'GEOIDFQ', 'SUMLEVEL', 'YEAR', 'METRIC', 'FUEL_TYPE', 'VALUE']]

In [21]:
juris_long['SUMLEVEL'] = [x[:3] for x in juris_long['GEOIDFQ']]
juris_long['METRIC'] = ['Facilities' if x.startswith('count_') else 'Capacity' for x in juris_long['FuelType']]
juris_long['FUEL_TYPE'] = [x.split("_")[-1] for x in juris_long['FuelType']]
juris_long = juris_long.rename(columns = {'JURISNAME':'NAME','Year':'YEAR', 'value':"VALUE"})
juris_long[['NAME', 'GEOIDFQ', 'SUMLEVEL', 'YEAR', 'METRIC', 'FUEL_TYPE', 'VALUE']]
juris_long['VALUE'] = [float(x) for x in juris_long['VALUE']]
juris_long = juris_long[['NAME', 'GEOIDFQ', 'SUMLEVEL', 'YEAR', 'METRIC', 'FUEL_TYPE', 'VALUE']]

In [22]:
all_long = pd.concat([region_long, county_long, juris_long])

In [23]:
all_long.head()

,NAME,GEOIDFQ,SUMLEVEL,YEAR,METRIC,FUEL_TYPE,VALUE
0,15-County Region,M010000US001,M01,2000,Facilities,Biofuel,0.0
1,15-County Region,M010000US001,M01,2001,Facilities,Biofuel,0.0
2,15-County Region,M010000US001,M01,2002,Facilities,Biofuel,0.0
3,15-County Region,M010000US001,M01,2003,Facilities,Biofuel,0.0
4,15-County Region,M010000US001,M01,2004,Facilities,Biofuel,0.0


In [24]:
all_long.to_csv('./output_data/morpc-renewenergyfacilities-der-long.csv', index=False)

In [25]:
frictionless.Schema.describe('./output_data/morpc-renewenergyfacilities-der-long.csv').to_yaml('./output_data/morpc-renewenergyfacilities-der-long.schema.yaml')

'fields:\n  - name: NAME\n    type: string\n  - name: GEOIDFQ\n    type: string\n  - name: SUMLEVEL\n    type: string\n  - name: YEAR\n    type: integer\n  - name: METRIC\n    type: string\n  - name: FUEL_TYPE\n    type: string\n  - name: VALUE\n    type: number\n'

In [26]:
description="""\
           A long for table containing renewable energy facilities that were approved by PUCO. See 
           https://puco.ohio.gov/utilities/electricity/resources/distributed-energy-generating-your-own-electricity
           Data pulled 2025-02-13 from https://maps.puco.ohio.gov/arcgis/rest/services/electric/Distributed_Energy_Resources/MapServer/0
           """
morpc.frictionless_create_resource('morpc-renewenergyfacilities-der-long.csv', 
                                   resourcePath='./output_data/morpc-renewenergyfacilities-der-long.resource.yaml',
                                   schemaPath='morpc-renewenergyfacilities-der-long.schema.yaml',
                                   description= " ".join(line.strip() for line in description.splitlines()),
                                   computeBytes=True,
                                   computeHash=True,
                                   writeResource=True
                                  )

morpc.frictionless_create_resource | INFO | Format not specified. Using format derived from data file extension: csv
morpc.frictionless_create_resource | INFO | Title not specified. Using placeholder value derived from data filename: morpc-renewenergyfacilities-der-long
morpc.frictionless_create_resource | INFO | Name not specified. Using placeholder value derived from data filename: morpc-renewenergyfacilities-der-long
morpc.frictionless_create_resource | WARNING | Resource type not specified. Using default type 'file'
morpc.frictionless_create_resource | INFO | Writing Frictionless Resource file to output_data\morpc-renewenergyfacilities-der-long.resource.yaml


{'name': 'morpc-renewenergyfacilities-der-long',
 'type': 'file',
 'title': 'morpc-renewenergyfacilities-der-long',
 'description': 'A long for table containing renewable energy facilities that '
                'were approved by PUCO. See '
                'https://puco.ohio.gov/utilities/electricity/resources/distributed-energy-generating-your-own-electricity '
                'Data pulled 2025-02-13 from '
                'https://maps.puco.ohio.gov/arcgis/rest/services/electric/Distributed_Energy_Resources/MapServer/0 ',
 'profile': 'data-resource',
 'path': 'morpc-renewenergyfacilities-der-long.csv',
 'scheme': 'file',
 'format': 'csv',
 'mediatype': 'text/csv',
 'hash': 'd80ae02c14b6e2b123327ff4c30ea4c2',
 'bytes': 6759051,
 'schema': 'morpc-renewenergyfacilities-der-long.schema.yaml'}